In [ ]:
import os
import time
import datetime
import schedule
import logging
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ✅ Setup Logging
LOG_FILE = "log.txt"
logging.basicConfig(
    filename=LOG_FILE,
    filemode='a',
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def log_print(message, level="info"):
    print(message)
    if level == "info":
        logging.info(message)
    elif level == "error":
        logging.error(message)
    elif level == "warning":
        logging.warning(message)

# ✅ Modify this path as needed
CHROMEDRIVER_PATH = os.path.join(os.getcwd(), 'chromedriver.exe')
BASE_DOWNLOAD_PATH = r"D:\F-Log report\LOG"

def create_daily_folder():
    now = datetime.datetime.now()
    date_folder = (now if now.hour >= 5 else now - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    folder_path = os.path.join(BASE_DOWNLOAD_PATH, date_folder)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

def get_driver(download_path):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless=new")  # optional
    chrome_options.add_argument("--allow-running-insecure-content")

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "safebrowsing.enabled": True
    })

    return webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)

def auto_login_and_download():
    download_dir = create_daily_folder()
    driver = get_driver(download_dir)
    wait = WebDriverWait(driver, 20)

    try:
        log_print("Opening portal...")
        driver.get("https://10.0.100.62:4444")

        # Login
        wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[1]"))).send_keys("admin")
        driver.find_element(By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[2]").send_keys("Ipac#2023#@!253")
        driver.find_element(By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[3]").click()
        log_print("✅ Logged in successfully.")

        # Navigate & download
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[1]/div[3]/nav/section[1]/ul/li[3]/a"))).click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[1]/div[3]/div/nav/ul/li[5]"))).click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[1]/div[1]/div[2]/label[1]"))).click()

        # Report settings
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[1]/td[2]/label[2]").click()
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[2]/td[2]/div[1]/label").click()
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[3]/td[2]/label[1]").click()

        # Search and download
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[10]/td[2]/input").click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[2]/div[2]/div[6]/div/div/label"))).click()

        log_print(f"⬇️  Download started. Folder: {download_dir}")
        time.sleep(10)

    except TimeoutException as te:
        log_print(f"⛔ Timeout: {te}", level="error")
    except Exception as e:
        log_print(f"⛔ Error: {e}", level="error")
    finally:
        driver.quit()
        log_print("✅ Task complete. Browser closed.")

# ✅ Schedule every 20 minutes
schedule.every(20).minutes.do(auto_login_and_download)

log_print("🚀 Scheduler started. Running every 20 minutes...")

while True:
    schedule.run_pending()
    time.sleep(1)


🕒 Scheduler started. Task will repeat every 20 minutes...


KeyboardInterrupt: 